In [8]:
import tensorly as tl
import torch
import torch.nn.functional as F
from torch import nn
import numpy as np
import statistics
import torch.optim

In [17]:
# data preprocessing 1 batch : (963,144)
temp1 = []
temp2 = []
temp3 = []

data1 = []
data2 = []
data3 = []
start = 0
indx = 0
with open("/home/seyunkim/Documents/time_series_data/PEMS-SF/PEMS_train") as f:
    for i, line in enumerate(f):
        if i==1:
            line = f.readline()
            line = line[1:len(line)-1]
            indx = line.find(';',indx,-1)
            temp1 = line[start:indx]
            data1 = np.asarray(temp1.split(' '))
            start = indx+1
            temp1 = []
            while line.find(';',start,-1)!=-1:
                indx = line.find(';',start,-1)
                temp1 = line[start:indx]
                tempnp = np.asarray(temp1.split(' '))
                data1 = np.vstack((tempnp, data1))
                temp1=[]
                start=indx+1
            temp1 = line[start:-1]
            data1 = np.vstack((data1,np.asarray(temp1.split(' '))))
        if i==2:
            start = 0
            indx = 0
            line = f.readline()
            line = line[1:len(line)-1]
            indx = line.find(';',indx,-1)
            temp2 = line[start:indx]
            data2 = np.asarray(temp2.split(' '))
            start = indx+1
            temp2 = []
            while line.find(';',start,-1)!=-1:
                indx = line.find(';',start,-1)
                temp2 = line[start:indx]
                tempnp = np.asarray(temp2.split(' '))
                data2 = np.vstack((tempnp, data2))
                temp2=[]
                start=indx+1
            temp2 = line[start:-1]
            data2 = np.vstack((data2,np.asarray(temp2.split(' '))))
        if i==3:
            start = 0
            indx = 0
            line = f.readline()
            line = line[1:len(line)-1]
            indx = line.find(';',indx,-1)
            temp3 = line[start:indx]
            data3 = np.asarray(temp3.split(' '))
            start = indx+1
            temp3 = []
            while line.find(';',start,-1)!=-1:
                indx = line.find(';',start,-1)
                temp3 = line[start:indx]
                tempnp = np.asarray(temp3.split(' '))
                data3 = np.vstack((tempnp, data3))
                temp3=[]
                start=indx+1
            temp3 = line[start:-1]
            data3 = np.vstack((data3,np.asarray(temp3.split(' '))))


In [19]:
train_tensor=torch.FloatTensor(np.ndarray.astype(data1,'float32').reshape(1,1,963,144))
val_tensor=torch.FloatTensor(np.ndarray.astype(data2,'float32').reshape(1,1,963,144))
test_tensor=torch.FloatTensor(np.ndarray.astype(data3,'float32').reshape(1,1,963,144))

In [2]:
# class Lambda(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.func = func
    
#     def forward(self, x):
#         return self.func(x)

In [3]:
# tl.set_backend('numpy')
# tensor_data = torch.FloatTensor(np.random.rand(1,1,30,500))
# split = tensor_data.shape[3]//4

In [4]:
# train_tensor = tensor_data[:,:,:,0:split*2]
# test_tensor = tensor_data[:,:,:,split*2:split*3]
# val_tensor = tensor_data[:,:,:,split*3:tensor_data.shape[3]]

In [20]:
class CNN_Extractor(nn.Module):
    def __init__(self, w, R):
        super().__init__()
        #self.local_kernel = local_kernel
        #self.global_kernel =  global_kernel
#         self.w = w
        self.R = R
        self.w = w
        self.conv2d = nn.Conv2d(1,1,(self.R,1))
        self.fc = nn.Linear(in_features=self.w, out_features=self.R) # Change this number of out_features to forecast longer time
        
    def forward(self, x):
        x = self.conv2d(x)
        x = self.fc(x)
        x = torch.tanh(x)
#         x = F.relu(x)
#         x = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=(self.R,2))(x)
#         x = nn.Linear(in_features=np.prod(x.shape), out_features=(2*self.R))(x)
#         x = F.relU(x)
        return x

In [26]:
# Hyperparameters
learning_rate = 1e-3
train_len = train_tensor.shape[3]
val_len = val_tensor.shape[3]
test_len = test_tensor.shape[3]
epochs = 100
w = 100
# Create Model
cnn_extractor = CNN_Extractor(w,963)
loss_fnc = nn.MSELoss()
optimizer = torch.optim.SGD(cnn_extractor.parameters(), lr=learning_rate, momentum=0.9)


In [27]:
# Train and evaluation
for epoch in range(epochs):
    losses = torch.tensor([0])
    for i in range(train_len-w):
        train_input = train_tensor[:,:,:,i:i+w] #input Rx1
        train_target = torch.transpose(train_tensor[:,:,:,i+w:i+w+1],2,3)
        
        cnn_extractor.train()
        y_pred = cnn_extractor(train_input)
        loss = loss_fnc(train_target, y_pred)
        losses = torch.hstack((losses, loss))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
#         if i%20==0:
#             print(f'i: {i}, \tWeight: {cnn_extractor.conv2d.weight.grad}')
    if epoch%2==0:
        cnn_extractor.eval()
        val_losses = torch.tensor([0])
        for i in range(val_len-w):
            val_input = val_tensor[:,:,:,i:i+w]
            val_target = torch.transpose(val_tensor[:,:,:,i+w:i+w+1],2,3)
            y_val = cnn_extractor(val_input)
            val_loss = loss_fnc(val_target, y_val)
            val_losses = torch.dstack((val_losses, val_loss))

        val_avg = torch.mean(val_losses)
        train_avg = torch.mean(losses)
        print(f'Epoch: {epoch}, Loss: {train_avg}\t\tEpoch: {epoch}, Val-loss: {val_avg}')



Epoch: 0, Loss: 0.008665638975799084		Epoch: 0, Val-loss: 0.006832281593233347
Epoch: 2, Loss: 0.00854423362761736		Epoch: 2, Val-loss: 0.006807421334087849
Epoch: 4, Loss: 0.00850010011345148		Epoch: 4, Val-loss: 0.006803241092711687
Epoch: 6, Loss: 0.008461720310151577		Epoch: 6, Val-loss: 0.006797481328248978
Epoch: 8, Loss: 0.008427277207374573		Epoch: 8, Val-loss: 0.006790291517972946
Epoch: 10, Loss: 0.008395512588322163		Epoch: 10, Val-loss: 0.0067817470990121365
Epoch: 12, Loss: 0.008365599438548088		Epoch: 12, Val-loss: 0.006771961692720652
Epoch: 14, Loss: 0.008336999453604221		Epoch: 14, Val-loss: 0.006761061958968639
Epoch: 16, Loss: 0.008309351280331612		Epoch: 16, Val-loss: 0.006749172229319811
Epoch: 18, Loss: 0.008282415568828583		Epoch: 18, Val-loss: 0.006736416835337877
Epoch: 20, Loss: 0.008256029337644577		Epoch: 20, Val-loss: 0.006722903810441494
Epoch: 22, Loss: 0.008230081759393215		Epoch: 22, Val-loss: 0.006708735134452581
Epoch: 24, Loss: 0.00820449274033308		E

In [8]:
# test_data = test_tensor[:,:,:,0:5] 
# prediction = cnn_extractor(test_data)
# test_target = test_tensor[:,:,:,5:6]

In [24]:
def test_evaluate(data, window, comment):
    test_losses = torch.tensor([0])
    for i in range(data.shape[3]-window):
        test_data = data[:,:,:,i:i+window]
        test_target = torch.transpose(data[:,:,:,i+window:i+window+1],2,3)
        prediction = cnn_extractor(test_data)

        test_loss = loss_fnc(test_target, prediction)
        test_losses = torch.dstack((test_loss, test_losses))
    
    test_avg = torch.mean(test_losses)
    print(f'Comment: {comment}\nTest Loss : {test_avg}')

In [28]:
test_evaluate(test_tensor, w, "w=100")

Comment: w=100
Test Loss : 0.006913019344210625


In [25]:
test_evaluate(test_tensor, w, "real data")

Comment: real data
Test Loss : 0.008898423984646797


In [16]:
test_evaluate(test_tensor, "fixed eval")

Comment: fixed eval
Test Loss : 0.08254538476467133


In [10]:
test_evaluate(test_tensor, "dawon input")

Comment: dawon input
Test Loss : 0.08254538476467133


In [11]:
test_evaluate(test_tensor, "larger window, no global")

Comment: larger window, no global
Test Loss : 0.08254538476467133


In [12]:
test_evaluate(test_tensor, "Larger dataset")

Comment: Larger dataset
Test Loss : 0.08254538476467133


In [13]:
test_evaluate(test_tensor, "Without ReLu")

Comment: Without ReLu
Test Loss : 0.08254538476467133


In [14]:
# # Evaluate with test data
# test_losses = torch.tensor([0])
# for i in range(test_len-5):
#     test_data = test_tensor[:,:,:,i:i+5]
#     test_target = torch.transpose(test_tensor[:,:,:,i+5:i+6],2,3)
#     prediction = cnn_extractor(test_data)
    
#     test_loss = loss_fnc(test_target, prediction)
#     test_losses = torch.dstack((test_loss, test_losses))

# test_avg = torch.mean(test_losses)
# print(f'Test Loss : {test_avg}')
    